In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable
import pandas as pd
from torch.utils.data import Dataset,DataLoader


In [2]:
class diabete_dataset(Dataset):
    
    def __init__(self):
        df=pd.read_csv('diabetes.csv',header=None,sep=',',dtype=np.float32)
        self.x_data=Variable(torch.from_numpy(df[[0, 1, 2, 3, 4, 5, 6, 7]].as_matrix()))
        self.y_data=Variable(torch.from_numpy(df[[8]].as_matrix()))
        self.len=df.shape[0]
                      
    def __getitem__(self,index):
        
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
                       
        return self.len

In [3]:
dataset_=diabete_dataset()

In [4]:
train_loader=DataLoader(dataset=dataset_,batch_size=64,shuffle=True)

In [5]:
data,label=next(iter(train_loader))

In [6]:
data.size()

torch.Size([64, 8])

In [7]:
class Network(torch.nn.Module):
    
    def __init__(self):
        super(Network,self).__init__()
        self.fc1=torch.nn.Linear(8,6)
        self.fc2=torch.nn.Linear(6,4)
        self.fc3=torch.nn.Linear(4,2)
        self.fc4=torch.nn.Linear(2,1)
        #self.sigmoid=torch.nn.Sigmoid()
        
        
    def forward(self,x):
        
        out1=F.relu(self.fc1(x))
        out2=F.relu(self.fc2(out1))
        out3=F.relu(self.fc3(out2))
        out4=F.sigmoid(self.fc4(out3))   #sof max at end, or sigmoid all the way did not work out
        y_pred=out4
        return y_pred
        

In [8]:
model=Network()

In [9]:
criterion=torch.nn.BCELoss(size_average=True)
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)  #had to change from SGD to Adam but later noy much difference

In [10]:
torch.set_printoptions(precision=2)
epochs=1500
loss_running=0
count=0
count_batch=0
sum_acc=0
for epoch in range(epochs):

    for input,label in iter(train_loader):
        
        y_pred=model(input)
        loss=criterion(y_pred,label)
        loss_running=loss_running+loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count=count+1
        y_pred_round=torch.round(y_pred)
        count_batch=count_batch+(label.size()[0])
  
    
        acc=(label==y_pred_round).sum().item()    #/label.size()[0]
        sum_acc=sum_acc+acc
    if(epoch%100==0): print('epoch',epoch,'/',epochs,'running_loss:',(loss_running/count_batch),'Accuracy',(sum_acc*100/count_batch))
    
    

epoch 0 / 1500 running_loss: tensor(1.00000e-02 *
       1.26) Accuracy 34.65085638998683
epoch 100 / 1500 running_loss: tensor(1.00000e-02 *
       1.05) Accuracy 58.641516325545595
epoch 200 / 1500 running_loss: tensor(1.00000e-03 *
       9.40) Accuracy 68.39517825890312
epoch 300 / 1500 running_loss: tensor(1.00000e-03 *
       8.77) Accuracy 71.91531084352115
epoch 400 / 1500 running_loss: tensor(1.00000e-03 *
       8.37) Accuracy 73.67910920984758
epoch 500 / 1500 running_loss: tensor(1.00000e-03 *
       8.10) Accuracy 74.75194538459313
epoch 600 / 1500 running_loss: tensor(1.00000e-03 *
       7.90) Accuracy 75.55238414675584
epoch 700 / 1500 running_loss: tensor(1.00000e-03 *
       7.74) Accuracy 76.13328597016496
epoch 800 / 1500 running_loss: tensor(1.00000e-03 *
       7.62) Accuracy 76.58378278798406
epoch 900 / 1500 running_loss: tensor(1.00000e-03 *
       7.51) Accuracy 76.9440776534344
epoch 1000 / 1500 running_loss: tensor(1.00000e-03 *
       7.42) Accuracy 77.2344